In [2]:
! pip install ollama wikipedia langchain langchain_community -q

### Herramienta de matemáticas

In [ ]:
evaluador_de_expresiones_tool = {
        'type': 'function',
        'function': {
          'name': 'evaluador_expresiones',
          'description': 'Evalúa una expresión matemática siguiendo el orden de operaciones básicas',
          'parameters': {
            'type': 'object',
            'properties': {
              'expression': {
                'type': 'string',
                'description': 'La expresión matemática a evaluar. La expresión puede incluir enteros, decimales, paréntesis y los operadores +, -, *, y /.',
              }
            },
            'required': ['expression'],
          },
        },
      }


In [4]:
def evaluador_expresiones(exp=""):
    try:
        return f"El resultado de {exp} es {eval(exp)}"
    except Exception as e:
        return str(e)

In [5]:
eval("5+7")

12

In [7]:
evaluador_expresiones("4*8/2")

'El resultado de 4*8/2 es 16.0'

### Herramienta de Wikipedia

In [8]:
wikipedia_tool = {
        'type': 'function',
        'function': {
          'name': 'wikipedia',
          'description': 'Un buscador de Wikipedia. Útil cuando necesitas responder preguntas generales sobre personas, lugares, empresas, hechos, eventos históricos u otros temas. La entrada debe ser una consulta de búsqueda. sieempre respondes en español',
          'parameters': {
            'type': 'object',
            'properties': {
              'query': {
                'type': 'string',
                'description': 'consulta para buscar en Wikipedia',
              }
            },
            'required': ['query'],
          },
        },
      }

In [9]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
wiki_api_wrapper = WikipediaAPIWrapper(lang="es")
wikipedia = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [11]:
wikipedia("Gabriela mistral")

/var/folders/6m/_j6t9n6177j9y_gdsz4fh3r40000gn/T/ipykernel_61842/840889017.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  wikipedia("Gabriela mistral")


'Page: Gabriela Mistral\nSummary: Lucila de María Godoy Alcayaga[2]\u200b (Vicuña, 7 de abril de 1889-Nueva York, 10 de enero de 1957),[3]\u200b conocida como Gabriela Mistral, fue una poetisa, diplomática, profesora y pedagoga chilena. Por su trabajo poético, recibió el Premio Nobel de Literatura en 1945,[4]\u200b cuando se convirtió en la primera mujer iberoamericana[n 1]\u200b y la segunda persona latinoamericana[n 2]\u200b en recibir un Premio Nobel.\nNacida en una familia de recursos modestos, se desempeñó como profesora en diversas escuelas y se convirtió en una importante pensadora respecto al papel de la educación pública y llegó a participar en la reforma del sistema educacional mexicano.[5]\u200b A partir de la década de 1920, Mistral tuvo una vida itinerante al desempeñarse como cónsul y representante en organismos internacionales en América y Europa.\nComo poeta, es una de las figuras más relevantes de la literatura chilena y latinoamericana. Entre sus obras destacan Desola

### Herramienta de Clima

In [12]:
weather_tool = {
      'type': 'function',
      'function': {
        'name': 'pronostico_clima',
        'description': 'Obtén el clima actual para una ciudad.',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'Nombre de la ciudad',
            },
          },
          'required': ['city'],
        },
      },
    }

In [13]:
import requests

In [14]:
def pronostico_clima(city):
    geocode_url = f"https://geocode.xyz/{city}?json=1"
    geocode_response = requests.get(geocode_url)
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        latitude = geocode_data['latt']
        longitude = geocode_data['longt']
        
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        weather_response = requests.get(weather_url)
        if weather_response.status_code == 200:
            weather_data = weather_response.json()
            return f"Wather of {city} today: {weather_data['current_weather']}"
            
        else:
            return ""
    else:
        return ""

In [16]:
pronostico_clima("new york")

"Wather of new york today: {'time': '2025-06-06T17:30', 'interval': 900, 'temperature': 29.0, 'windspeed': 18.0, 'winddirection': 160, 'is_day': 1, 'weathercode': 0}"

In [21]:
import ollama
tool_history = []
def tool_calling(query):
    
    messages = [{'role': 'user', 'content': query}]

    response = ollama.chat(
        model='llama3.2:latest',
        messages=messages,
        tools=[evaluador_de_expresiones_tool, weather_tool, wikipedia_tool],
    )
    print(response['message'])
    
    messages.append(response['message'])

    if not response['message'].get('tool_calls'):
        print("El modelo no utilizó la función. Su respuesta fue:")
        print(response['message']['content'])

    if response['message'].get('tool_calls'):
        available_functions = {
            'evaluador_expresiones' : evaluador_expresiones,
            'wikipedia' : wikipedia,
            'pronostico_clima' : pronostico_clima
        }
    for tool in response['message']['tool_calls']:

        function_to_call = available_functions[tool['function']['name']]
        args = tool['function']['arguments'].values()
        function_response = function_to_call(*args)

        print("\nTool Response:\n", function_response)

        messages.append({'role': 'tool', 'content': function_response})

    final_response = ollama.chat(model='llama3.2:latest', messages=messages)

    return final_response['message']['content']

In [22]:
query = '¿Cuánto es (100+2)**3?'
result = tool_calling(query)
print("\n\nRespuesta:\n", result)

role='assistant' content='' thinking=None images=None tool_calls=[ToolCall(function=Function(name='evaluador_expresiones', arguments={'expression': '(100+2)^3'}))]

Tool Response:
 El resultado de (100+2)^3 es 101


Respuesta:
 Para calcular el resultado, primero debemos seguir la orden de operaciones (PEMDAS):

1. Evaluar la expresión dentro del paréntesis: 100 + 2 = 102
2. Elevar al cubo: 102^3 = 1061208

Por lo tanto, el resultado final es: 1061208


In [23]:
query = '¿Cuál es el pronóstico del clima para new york?'
result = tool_calling(query)
print("\n\nRespuesta:\n", result)

role='assistant' content='' thinking=None images=None tool_calls=[ToolCall(function=Function(name='pronostico_clima', arguments={'city': 'new york'}))]

Tool Response:
 Wather of new york today: {'time': '2025-06-06T17:30', 'interval': 900, 'temperature': 29.0, 'windspeed': 18.0, 'winddirection': 160, 'is_day': 1, 'weathercode': 0}


Respuesta:
 Lo siento, pero no tengo acceso a información en tiempo real o futura sobre el clima. Sin embargo, puedo proporcionarte un pronóstico general para New York.

El verano es la temporada más cálida y húmeda de New York, con temperaturas promedio que variaban entre 26°C y 32°C (79°F-90°F) durante los meses de junio a agosto. Sin embargo, las temperaturas pueden variar dependiendo de la ubicación dentro de la ciudad.

En invierno, New York experimenta climas fríos y nevados, con temperaturas promedio que variaban entre -2°C y 4°C (28°F-39


In [28]:
consulta = '¿Quién es Gabriel García Márquez?'
resultado = tool_calling(consulta)
print("\n\nRespuesta:\n", resultado)

role='assistant' content='' thinking=None images=None tool_calls=[ToolCall(function=Function(name='wikipedia', arguments={'query': 'Gabriel García Márquez'}))]

Tool Response:
 Page: Gabriel García Márquez
Summary: Gabriel José García Márquez (Aracataca, Magdalena, 6 de marzo de 1927-Ciudad de México, 17 de abril de 2014)[nota 1]​[2]​ fue un escritor, guionista y periodista colombiano. Reconocido por sus novelas y cuentos, también escribió narrativa de no ficción, discursos, reportajes, críticas cinematográficas y memorias. Estudió derecho y periodismo en la Universidad Nacional de Colombia e inició sus colaboraciones periodísticas en el diario El Espectador. Fue conocido como Gabo, o Gabito, por sus familiares y amigos.[3]​[4]​ En 1982 recibió el Premio Nobel de Literatura[5]​ «por sus novelas e historias cortas, en las que lo fantástico y lo real se combinan en un mundo compuesto de imaginación, lo que refleja la vida y los conflictos de un continente».[6]​[7]​ 
Junto a Julio Cortáza